In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
!gdown --id '1fpmRXRYFEvuujjxHTm05lFLLI-8_T-Y5' --output level-6.csv

Downloading...
From: https://drive.google.com/uc?id=1fpmRXRYFEvuujjxHTm05lFLLI-8_T-Y5
To: /content/level-6.csv
100% 183k/183k [00:00<00:00, 54.7MB/s]


In [3]:
train = pd.read_csv("level-6.csv")
train.head()

,index,Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium,Bacteria;Acidobacteriota;Acidobacteriae;Subgroup_2;__;__,Bacteria;Actinobacteria;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Acidimicrobiia;Microtrichales;Ilumatobacteraceae;uncultured,Bacteria;Actinobacteriota;Acidimicrobiia;uncultured;uncultured_forest_soil_bacterium;,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;F0332,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Varibaculum,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Bifidobacterium,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Gardnerella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Corynebacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Lawsonella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Dietziaceae;Dietzia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Mycobacteriaceae;Mycobacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Gordonia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Rhodococcus,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Williamsia,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Blastococcus,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Modestobacter,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Sporichthyaceae;uncultured,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;__;__,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;Actinomycetales_bacterium,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;__,Bacteria;Actinobacteriota;Actinobacteria;Kineosporiales;Kineosporiaceae;Quadrisphaera,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Beutenbergiaceae;Miniimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Bogoriellaceae;Georgenia,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Brevibacteriaceae;Brevibacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Brachybacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Dermabacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Barrientosiimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Dermacoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Janibacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Lapillicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinimicrobium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Serinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;__,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Agrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Amnibacterium,...,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Oceanospirillales;Halomonadaceae;Halomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Orbales;Orbaceae;Gilliamella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Haemophilus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Pasteurella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;__,Bacteria;Proteobacteria;Gammaproteob

In [4]:
shortrun = train[train['merged'] < 19250].index
train = train.drop(shortrun,axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 167
Columns: 371 entries, index to Clinical
dtypes: float64(368), int64(1), object(2)
memory usage: 377.8+ KB


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 167
Columns: 371 entries, index to Clinical
dtypes: float64(368), int64(1), object(2)
memory usage: 377.8+ KB


In [6]:
CAT_COL = ["index", "Clinical"]

NUM_COL=[]
for i in range(len(train.columns)):
  NUM_COL.append(train.columns[i])
NUM_COL.remove('index')
NUM_COL.remove('Clinical')

cat_col = []
num_col = []
for col in train:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)
for col in cat_col:
  train[col] = train[col].astype(str)

df_cat = train.loc[:,cat_col] # take all the categorical columns
df_cat = pd.get_dummies(df_cat) # one hot encoding

df_num = train.loc[:,num_col] # take all the numerical columns
df_final = pd.concat([df_cat, df_num], axis=1) # concat categorical/numerical data

In [7]:
df_final.head()

,index_SRR10717942,index_SRR10717943,index_SRR10717944,index_SRR10717945,index_SRR10717947,index_SRR10717950,index_SRR10717951,index_SRR10717952,index_SRR10717953,index_SRR10717954,index_SRR10717956,index_SRR10717957,index_SRR10717958,index_SRR10717959,index_SRR10717960,index_SRR10717961,index_SRR10717963,index_SRR10717964,index_SRR10717965,index_SRR10717966,index_SRR10717967,index_SRR10717968,index_SRR10717969,index_SRR10717970,index_SRR10717971,index_SRR10717972,index_SRR10717973,index_SRR10717974,index_SRR10717975,index_SRR10717976,index_SRR10717977,index_SRR10717978,index_SRR10717979,index_SRR10717980,index_SRR10717981,index_SRR10717983,index_SRR10717984,index_SRR10717985,index_SRR10717986,index_SRR10717988,...,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Yersiniaceae;Serratia,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Oceanospirillales;Halomonadaceae;Halomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Orbales;Orbaceae;Gilliamella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Haemophilus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Pasteurella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Acinetobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Alkanindiges,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Enhydrobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Moraxella,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Psychrobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;uncultured,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Pseudomonadaceae;Pseudomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;__;__,Bacteria;Proteobacteria;Gammaproteobacteria;Salinisphaerales;Solimonadaceae;Nevskia,Bacteria;Proteobacteria;Gammaproteobacteria;Thiotrichales;Thiotrichaceae;Thiothrix,Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Photobacterium,Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Vibrio,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Rhodanobacteraceae;Ahniella,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Rhodanobacteraceae;Pseudofulvimonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Rhodanobacteraceae;Rhodanobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Luteimonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Lysobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Pseudoxanthomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Stenotrophomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Xanthomonas,Bacteria;Proteobacteria;Gammaproteobacteria;__;__;__,Bacteria;Proteobacteria;__;__;__;__,Bacteria;SAR324_clade(Marine_group_B);uncultured_bacterium;;__;__,Bacteria;Spirochaetota;Spirochaetia;Spirochaetales;Spirochaetaceae;Treponema,Bacteria;Synergistota;Synergistia;Synergistales;Synergistaceae;Fretibacterium,Bacteria;Verrucomicrobiota;Chlamydiae;Chlamydiales;__;__,Bacteria;Verrucomicrobiota;Omnitrophia;Omnitrophales;Omnitrophaceae;Candidatus_Omnitrophus,Bacteria;__;__;__;__;__,Unassigned;__;__;__;__;__,merged
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.060593,0.0,0.0,0.000477,0.0,0.084239,0.0,0.0,0.0,0.0,0.013825,0.0,0.0,0.0,0.0,

In [8]:
not_select = ["index", "Clinical", "merged"]
train_select = train.drop(not_select,axis=1)
train_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 167
Columns: 368 entries, Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium to Unassigned;__;__;__;__;__
dtypes: float64(368)
memory usage: 374.8 KB


In [9]:
cat_col = []
num_col = []
for col in train_select:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)

for col in cat_col:
  if train_select[col].dtype != "O":
    # print(col)
    train_select[col] = train_select[col].astype(str)

df_cat_select = train_select.loc[:,cat_col] # take all the categorical columns
#df_cat_select = pd.get_dummies(df_cat_select) # one hot encoding

df_num_select = train_select.loc[:,num_col] # take all the numerical columns
df_final_select = pd.concat([df_cat_select, df_num_select], axis=1) # concat categorical/numerical data

In [10]:
df_final_select.head()

,Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium,Bacteria;Acidobacteriota;Acidobacteriae;Subgroup_2;__;__,Bacteria;Actinobacteria;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Acidimicrobiia;Microtrichales;Ilumatobacteraceae;uncultured,Bacteria;Actinobacteriota;Acidimicrobiia;uncultured;uncultured_forest_soil_bacterium;,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;F0332,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Varibaculum,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Bifidobacterium,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Gardnerella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Corynebacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Lawsonella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Dietziaceae;Dietzia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Mycobacteriaceae;Mycobacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Gordonia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Rhodococcus,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Williamsia,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Blastococcus,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Modestobacter,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Sporichthyaceae;uncultured,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;__;__,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;Actinomycetales_bacterium,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;__,Bacteria;Actinobacteriota;Actinobacteria;Kineosporiales;Kineosporiaceae;Quadrisphaera,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Beutenbergiaceae;Miniimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Bogoriellaceae;Georgenia,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Brevibacteriaceae;Brevibacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Brachybacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Dermabacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Barrientosiimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Dermacoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Janibacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Lapillicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinimicrobium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Serinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;__,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Agrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Amnibacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Curtobacterium,...,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Yersiniaceae;Serratia,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Oceanospirillales;Halomonadaceae;Halomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Orbales;Orbaceae;Gilliamella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pas

#DNN

In [11]:
import os
import numpy as np
import keras as K
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
os.environ['TF_CPP_MIN_LOG_LEVEL']= '2'

In [12]:
target_var = 'Clinical'
features = list(df_final_select.columns)
# 目標變量的類別
Class = train[target_var].unique()
# 目標變量的類別字典
Class_dict = dict(zip(Class, range(len(Class))))
# 增加一列target, 將目標變量進行編碼
train['target'] = train[target_var].apply(lambda x: Class_dict[x])
# 對目標變量進行0-1編碼(One-hot Encoding)
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
transformed_labels = lb.transform(train['target'])
y_bin_labels = []  # 對多分類進行0-1編碼的變量
for i in range(transformed_labels.shape[1]):
    y_bin_labels.append( 'y' + str(i))
    train['y' + str(i)] = transformed_labels[:, i]
# 將數據集分為訓練集和測試集
train_x, test_x, train_y, test_y = train_test_split(df_final_select[features], train[y_bin_labels], train_size=0.6, test_size=0.4, random_state=0)

In [13]:
np.random.seed(5)
# model
init = K.initializers.glorot_uniform(seed=1)
optimizer = K.optimizers.Adam()
model = K.models.Sequential()
model.add(K.layers.Dense(units=1024, input_dim=368, kernel_initializer=init, activation='relu'))
model.add(K.layers.Dense(units=2048, kernel_initializer=init, activation= 'relu'))
model.add(K.layers.Dense(units=1024, kernel_initializer=init, activation= 'relu'))
model.add(K.layers.Dense(units=3, kernel_initializer=init, activation='relu'))
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [14]:
# 訓練模型
b_size = 1
max_epochs = 50
print("Starting training ")
h = model.fit(train_x, train_y, batch_size=b_size, epochs=max_epochs, shuffle=True, verbose=1)
print("Training finished \n")

Starting training 
Epoch 1/50
78/78 [==============================] - 15s 3ms/step - loss: 9.6706 - accuracy: 0.4002
Epoch 2/50
78/78 [==============================] - 0s 3ms/step - loss: 11.1653 - accuracy: 0.3073
Epoch 3/50
78/78 [==============================] - 0s 3ms/step - loss: 11.1625 - accuracy: 0.3075
Epoch 4/50
78/78 [==============================] - 0s 3ms/step - loss: 11.3279 - accuracy: 0.2972
Epoch 5/50
78/78 [==============================] - 0s 3ms/step - loss: 10.3008 - accuracy: 0.3609
Epoch 6/50
78/78 [==============================] - 0s 3ms/step - loss: 10.5479 - accuracy: 0.3456
Epoch 7/50
78/78 [==============================] - 0s 3ms/step - loss: 9.8747 - accuracy: 0.3874
Epoch 8/50
78/78 [==============================] - 0s 2ms/step - loss: 9.8764 - accuracy: 0.3872
Epoch 9/50
78/78 [==============================] - 0s 3ms/step - loss: 9.6261 - accuracy: 0.4028
Epoch 10/50
78/78 [==============================] - 0s 3ms/step - loss: 8.9092 - accuracy: 0

In [15]:
eval = model.evaluate(test_x, test_y, verbose=0)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100))

Evaluation on test data: loss = 10.848718 accuracy = 32.69% 



#PH-CNN
##reference:
###https://github.com/dfioravanti/phylogenetic-cnn
###https://www.tensorflow.org/tutorials/images/cnn

In [16]:
# import pandas as pd
# import numpy as np
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# sys.path.append('/content/drive/My Drive/Colab Notebooks/phylogenetic-cnn-master')

In [17]:
import os
from tensorflow.keras import datasets, layers, models

# from phcnn.layers import PhyloConv1D, euclidean_distances
# from keras.utils.np_utils import to_categorical

# from keras.layers import (Lambda, MaxPooling1D, Flatten, Dropout, Dense, Input)
# from keras.models import Model
# from keras.backend import floatx

##Import coordinate data

In [18]:
#reshape
!gdown --id '1dBimDrlyC6eHRcPbCIuj7PzgAwJ-VSby' --output ordinate.csv
C = pd.read_csv("ordinate.csv")
# Xs = df_final_select.values

Downloading...
From: https://drive.google.com/uc?id=1dBimDrlyC6eHRcPbCIuj7PzgAwJ-VSby
To: /content/ordinate.csv
100% 207k/207k [00:00<00:00, 75.0MB/s]


In [19]:
# not_select = ['SRR10717998', 'SRR10718078', 'SRR10718037', 'SRR10718080', 'SRR10717987', 'SRR10718039', 'SRR10718095', 'SRR10718000',
#  'SRR10718082', 'SRR10718060', 'SRR10718016', 'SRR10717949', 'SRR10718052', 'SRR10718067', 'SRR10717990', 'SRR10718001',
#  'SRR10717993', 'SRR10717955','SRR10718024', 'SRR10718096', 'SRR10718044', 'SRR10717946', 'SRR10718092', 'SRR10718047',
#  'SRR10717948', 'SRR10718089', 'SRR10718097','SRR10718055', 'SRR10718013', 'SRR10718076', 'SRR10718014', 'SRR10718049',
#  'SRR10717962', 'SRR10717982', 'SRR10718071', 'SRR10717991', 'SRR10718075', 'SRR10718073']
shortrun = C[C['merged'] < 19250].index
C = C.drop(shortrun,axis=0)
C.head()

,index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,merged
0,SRR10717942,-0.148863,-0.015859,-0.182425,0.036586,-0.060318,-0.010474,-0.073774,0.022891,0.045670,0.009416,0.003135,-0.048164,-0.043423,0.046419,-0.015531,0.016605,0.067995,-0.019294,0.002615,-0.063570,-0.008053,0.053429,-0.005841,-0.032327,0.000089,0.026441,-0.003692,0.000019,0.043579,-0.021603,0.006190,0.003795,0.025930,-0.039945,-0.036880,-0.012962,0.046948,0.017464,-0.025889,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23645
1,SRR10717943,-0.155874,-0.015489,-0.166780,0.040794,-0.040257,0.015205,0.039951,-0.042126,-0.024423,0.016980,-0.015751,0.007062,-0.029513,-0.017791,0.047194,0.018195,0.002031,0.030278,-0.037003,-0.001940,0.022938,-0.013325,0.061446,0.015850,-0.005763,-0.008191,-0.023836,0.002106,-0.020018,-0.002994,-0.013115,-0.038409,0.016884,0.010377,0.013561,-0.013599,-0.019550,-0.033015,-0.032819,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22937
2,SRR10717944,-0.212688,0.126506,0.136332,0.149565,-0.107294,0.068687,-0.042806,-0.023720,0.053022,0.052038,0.067566,0.025259,-0.044514,0.037947,0.016528,-0.044443,-0.037207,0.065178,0.010344,-0.019677,-0.013522,-0.012546,0.010742,-0.023155,0.027363,-0.038568,-0.011360,0.009473,-0.012771,-0.001583,-0.003747,0.026933,-0.036237,0.033401,0.007992,-0.034118,0.002309,0.028903,0.007183,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23355
3,SRR10717945,-0.036509,-0.026844,0.033635,0.126231,0.113269,0.034193,0.020957,-0.012934,0.042122,-0.020769,0.045778,-0.063197,0.003277,-0.043442,0.001005,0.032437,0.004249,0.014258,0.005888,0.029863,-0.007705,0.050286,0.028003,0.015712,0.001625,0.004379,-0.053675,0.011616,0.015696,0.006473,-0.006499,0.024532,-0.015546,-0.007611,-0.001774,0.056346,0.024639,0.017048,0.023227,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19504
5,SRR10717947,-0.171395,-0.068856,-0.069648,-0.037126,0.094986,0.026219,0.055954,0.030454,-0.074112,0.052688,-0.018514,-0.004871,-0.034680,-0.025276,0.009162,-0.044719,0.005188,-0.002254,0.000137,-0.035925,-0.039550,-0.032502,0.016004,-0.011044,-0.022212,0.020685,0.035681,0.011047,-0.014573,-0.039898,-0.013890,0.000235,0.017508,0.023251,-0.020433,-0.006334,0.002924,0.000900,0.017421,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29110


In [20]:
not_select=['index', 'merged']
C_final = C.drop(not_select,axis=1)
C_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 167
Columns: 168 entries, 1 to 168
dtypes: float64(84), int64(84)
memory usage: 171.6 KB


In [21]:
C_final.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168
0,-0.148863,-0.015859,-0.182425,0.036586,-0.060318,-0.010474,-0.073774,0.022891,0.045670,0.009416,0.003135,-0.048164,-0.043423,0.046419,-0.015531,0.016605,0.067995,-0.019294,0.002615,-0.063570,-0.008053,0.053429,-0.005841,-0.032327,0.000089,0.026441,-0.003692,0.000019,0.043579,-0.021603,0.006190,0.003795,0.025930,-0.039945,-0.036880,-0.012962,0.046948,0.017464,-0.025889,0.018509,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-0.155874,-0.015489,-0.166780,0.040794,-0.040257,0.015205,0.039951,-0.042126,-0.024423,0.016980,-0.015751,0.007062,-0.029513,-0.017791,0.047194,0.018195,0.002031,0.030278,-0.037003,-0.001940,0.022938,-0.013325,0.061446,0.015850,-0.005763,-0.008191,-0.023836,0.002106,-0.020018,-0.002994,-0.013115,-0.038409,0.016884,0.010377,0.013561,-0.013599,-0.019550,-0.033015,-0.032819,-0.065783,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.212688,0.126506,0.136332,0.149565,-0.107294,0.068687,-0.042806,-0.023720,0.053022,0.052038,0.067566,0.025259,-0.044514,0.037947,0.016528,-0.044443,-0.037207,0.065178,0.010344,-0.019677,-0.013522,-0.012546,0.010742,-0.023155,0.027363,-0.038568,-0.011360,0.009473,-0.012771,-0.001583,-0.003747,0.026933,-0.036237,0.033401,0.007992,-0.034118,0.002309,0.028903,0.007183,-0.023905,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-0.036509,-0.026844,0.033635,0.126231,0.113269,0.034193,0.020957,-0.012934,0.042122,-0.020769,0.045778,-0.063197,0.003277,-0.043442,0.001005,0.032437,0.004249,0.014258,0.005888,0.029863,-0.007705,0.050286,0.028003,0.015712,0.001625,0.004379,-0.053675,0.011616,0.015696,0.006473,-0.006499,0.024532,-0.015546,-0.007611,-0.001774,0.056346,0.024639,0.017048,0.023227,0.018659,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,-0.171395,-0.068856,-0.069648,-0.037126,0.094986,0.026219,0.055954,0.030454,-0.074112,0.052688,-0.018514,-0.004871,-0.034680,-0.025276,0.009162,-0.044719,0.005188,-0.002254,0.000137,-0.035925,-0.039550,-0.032502,0.016004,-0.011044,-0.022212,0.020685,0.035681,0.011047,-0.014573,-0.039898,-0.013890,0.000235,0.017508,0.023251,-0.020433,-0.006334,0.002924,0.000900,0.017421,-0.040708,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
# X1 = df_final_select.values
# X2 = C_final.values
# X_final = np.concatenate((X1, X2),axis=1)
X_final = pd.concat([df_final_select, C_final], axis=1)
X_final.shape

(130, 536)

In [23]:
target_var = 'Clinical'
features = list(X_final.columns)
# 目標變量的類別
Class = train[target_var].unique()
# 目標變量的類別字典
Class_dict = dict(zip(Class, range(len(Class))))
# 增加一列target, 將目標變量進行編碼
train['target'] = train[target_var].apply(lambda x: Class_dict[x])
# 對目標變量進行0-1編碼(One-hot Encoding)
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
transformed_labels = lb.transform(train['target'])
y_bin_labels = []  # 對多分類進行0-1編碼的變量
for i in range(transformed_labels.shape[1]):
    y_bin_labels.append( 'y' + str(i))
    train['y' + str(i)] = transformed_labels[:, i]
# 將數據集分為訓練集和測試集
train_x, test_x, train_y, test_y = train_test_split(X_final[features], train[y_bin_labels], train_size=0.6, test_size=0.4, random_state=0)

In [69]:
# nb_coordinates = C_final.shape[0]
# print('nb_coordinates=',nb_coordinates)
# Coords = np.empty((Xs.shape[0],) + C_final.shape, dtype=np.float64)
# # print('Xs.shape[0],=',(Xs.shape[0],))
# # print('Coords=',Coords)
# for i in range(Xs.shape[0]):
#     Coords[i] = C_final
# # add last dimension, i.e. channel, necessary for the Convolution
# # operator to work.
# Coords = np.expand_dims(Coords, axis=-1)
# Xs = np.expand_dims(Xs, axis=-1)
# print(Coords.shape, Xs.shape)

# train_x = train_x.values.reshape(78, 2, 2, 134)
train_x = train_x.reshape(78, 2, 2, 134)
# train_x = train_x.reshape(78, 67, 8, 1)
# test_x = test_x.values.reshape(52, 2, 2, 134)
test_x = test_x.reshape(52, 2, 2, 134)
# test_x = test_x.reshape(52, 67, 8, 1)

In [25]:
# nb_neighbors = 4
# nb_filters = 4

In [26]:
# data = Input(shape=(nb_features, 1), name="data", dtype=floatx())
# coordinates = Input(shape=(nb_coordinates, nb_features, 1), name="coordinates", dtype=floatx())
# conv_layer = data
# # We remove the padding that we added to work around keras limitations
# conv_crd = Lambda(lambda c: c[0], output_shape=lambda s: (s[1:]))(coordinates)
# print(conv_crd)
# distances = euclidean_distances(conv_crd)
# conv_layer, conv_crd = PhyloConv1D(distances, nb_neighbors, nb_filters, activation='relu')([conv_layer, conv_crd])

# max = MaxPooling1D(pool_size=2, padding="valid")(conv_layer)
# flatt = Flatten()(max)
# drop = Dropout(0.25)(Dense(units=64, activation='relu')(flatt))
# output = Dense(units=2, kernel_initializer="he_normal",
#                activation="softmax", name='output')(drop)

# model = Model(inputs=[data, coordinates], outputs=output)
# model.compile(optimizer='Adam', loss='categorical_crossentropy')

## Model(Same as paper)

In [82]:
model = K.models.Sequential()
model.add(K.layers.Conv2D(134, (1, 1), activation='relu', input_shape=train_x.shape[1:]))
model.add(K.layers.Conv2D(67, (1, 1), activation='relu', input_shape=train_x.shape[1:]))
model.add(K.layers.MaxPooling2D((2, 2)))
model.add(K.layers.Flatten())
model.add(K.layers.Dense(64, activation= 'relu'))
model.add(K.layers.Dropout(0.25))
model.add(K.layers.Dense(3,activation='sigmoid'))
# model.add(K.layers.Conv2D(64, (1, 1), activation='sigmoid'))
# model.add(K.layers.MaxPooling2D((2, 2)))
# model.add(K.layers.Conv2D(64, (3, 3), activation='relu'))

In [74]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 2, 2, 134)         18090     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 2, 2, 67)          9045      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 1, 1, 67)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 67)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                4352      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 3)               

In [83]:
opt = K.optimizers.Adam(learning_rate=0.0005)
# opt = K.optimizers.Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy',
        metrics=['accuracy'])

h2 = model.fit(train_x, train_y, epochs=15, batch_size=1,
      validation_data=(test_x, test_y))

Epoch 1/15
78/78 [==============================] - 2s 9ms/step - loss: 1.0987 - accuracy: 0.3463 - val_loss: 1.1050 - val_accuracy: 0.2692
Epoch 2/15
78/78 [==============================] - 0s 4ms/step - loss: 1.0901 - accuracy: 0.4287 - val_loss: 1.1086 - val_accuracy: 0.2115
Epoch 3/15
78/78 [==============================] - 0s 5ms/step - loss: 1.0821 - accuracy: 0.4529 - val_loss: 1.1170 - val_accuracy: 0.2308
Epoch 4/15
78/78 [==============================] - 0s 4ms/step - loss: 1.0678 - accuracy: 0.4240 - val_loss: 1.1136 - val_accuracy: 0.2692
Epoch 5/15
78/78 [==============================] - 0s 4ms/step - loss: 1.0501 - accuracy: 0.4928 - val_loss: 1.1189 - val_accuracy: 0.2885
Epoch 6/15
78/78 [==============================] - 0s 4ms/step - loss: 1.0276 - accuracy: 0.5159 - val_loss: 1.1191 - val_accuracy: 0.2885
Epoch 7/15
78/78 [==============================] - 0s 4ms/step - loss: 0.9840 - accuracy: 0.6760 - val_loss: 1.1228 - val_accuracy: 0.2692
Epoch 8/15
78/78 [==

##Model 2
Activation = relu  
No Dropout

In [115]:
model = K.models.Sequential()
model.add(K.layers.Conv2D(134, (1, 1), activation='relu', input_shape=train_x.shape[1:]))
model.add(K.layers.Conv2D(67, (1, 1), activation='relu', input_shape=train_x.shape[1:]))
model.add(K.layers.MaxPooling2D((2, 2)))
model.add(K.layers.Flatten())
model.add(K.layers.Dense(128, activation= 'relu'))
model.add(K.layers.Dropout(0.15))
model.add(K.layers.Dense(3,activation='relu'))

In [99]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 2, 2, 134)         18090     
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 2, 2, 67)          9045      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 1, 1, 67)          0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 67)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 128)               8704      
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 3)               

In [116]:
opt = K.optimizers.Adam(learning_rate=0.0003, decay=1e-3)
model.compile(optimizer=opt, loss='categorical_crossentropy',
        metrics=['accuracy'])

h2 = model.fit(train_x, train_y, epochs=15, batch_size=1,
      validation_data=(test_x, test_y))

Epoch 1/15
78/78 [==============================] - 1s 8ms/step - loss: 5.8095 - accuracy: 0.3002 - val_loss: 1.2637 - val_accuracy: 0.2500
Epoch 2/15
78/78 [==============================] - 0s 5ms/step - loss: 1.5552 - accuracy: 0.3877 - val_loss: 1.1752 - val_accuracy: 0.2885
Epoch 3/15
78/78 [==============================] - 0s 4ms/step - loss: 1.0654 - accuracy: 0.4699 - val_loss: 1.1302 - val_accuracy: 0.2885
Epoch 4/15
78/78 [==============================] - 0s 5ms/step - loss: 0.9421 - accuracy: 0.6129 - val_loss: 1.1525 - val_accuracy: 0.2692
Epoch 5/15
78/78 [==============================] - 0s 4ms/step - loss: 0.8539 - accuracy: 0.5876 - val_loss: 1.7901 - val_accuracy: 0.2308
Epoch 6/15
78/78 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.7582 - val_loss: 3.7191 - val_accuracy: 0.2308
Epoch 7/15
78/78 [==============================] - 0s 4ms/step - loss: 0.9261 - accuracy: 0.3896 - val_loss: 1.3072 - val_accuracy: 0.2308
Epoch 8/15
78/78 [=====